In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Load the dataset
data = pd.read_csv('/content/tesla.csv')
data = data.dropna()

In [4]:
# Create features and labels
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

<ipython-input-4-6078241aa6e9>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


In [5]:
# Create lagged features
for lag in range(1, 11):
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

In [6]:
# Add more technical indicators as features
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = data['Close'].diff().rolling(window=14).apply(lambda x: np.mean(np.where(x > 0, x, 0)) / (np.mean(np.abs(x)) + 1e-10), raw=True)
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()

In [7]:
data.dropna(inplace=True)

In [8]:
# Prepare feature and target arrays
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'MA10', 'MA50', 'RSI', 'EMA10', 'EMA50'] + [f'Close_lag_{lag}' for lag in range(1, 11)]
X = data[features]
y = data['Close']

In [9]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# Initialize the XGBoost Regressor
xgbr = xgb.XGBRegressor()

In [12]:
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

In [13]:
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(xgbr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.7, 0.8, 0.9]},
             scoring='neg_mean_squared_error')

In [14]:
# Get the best estimator
best_xgbr = grid_search.best_estimator_

In [15]:
# Make predictions on the testing set
y_pred = best_xgbr.predict(X_test)

In [16]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [17]:
# Calculate a custom accuracy metric
def custom_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs((y_true - y_pred) / y_true) <= tolerance)

accuracy = custom_accuracy(y_test, y_pred)

In [18]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error (MAE): 0.5796376855226887
Mean Squared Error (MSE): 1.1998657476299974
R-squared (R2): 0.9999097130170648
Custom Accuracy: 99.30%


In [19]:
# Make a prediction on new data
new_data = pd.DataFrame({
    'Open': [300],
    'High': [310],
    'Low': [295],
    'Adj Close': [305],
    'Volume': [1000000],
    'MA10': [305],
    'MA50': [310],
    'RSI': [0.01],
    'EMA10': [305],
    'EMA50': [310],
    'Close_lag_1': [298],
    'Close_lag_2': [299],
    'Close_lag_3': [297],
    'Close_lag_4': [296],
    'Close_lag_5': [295],
    'Close_lag_6': [294],
    'Close_lag_7': [293],
    'Close_lag_8': [292],
    'Close_lag_9': [291],
    'Close_lag_10': [290]
})

In [20]:
new_data_scaled = scaler.transform(new_data)
new_prediction = best_xgbr.predict(new_data_scaled)
print(f'Predicted Close Price: {new_prediction[0]}')

Predicted Close Price: 304.4063720703125
